---
title: "Horários do Curso de Engenharia de Materiais"
subtitle: "Semestre 2025/2"
format:
  html:
    toc: false
    toc-depth: 2
    theme: cosmo
    number-sections: false
    page-layout: full
    code-tools: false
    echo: false
    embed-resources: true
jupyter: python3
---

<div style="display: flex; align-items: center; gap: 2em; margin-bottom: 1em;">
  <img src="https://jcconcursos.com.br/media/_versions/orgao/concurso-ufpel-72815_sm.jpg" alt="Logotipo da UFPel" style="height: 80px;">
  <img src="https://wp.ufpel.edu.br/engmateriais/files/2025/02/engmat.png" alt="Logotipo do Curso de Engenharia de Materiais" style="height: 80px;">
</div>

**Universidade Federal de Pelotas (UFPel)**  
**Centro de Desenvolvimento Tecnológico (CDTec)**  
**Curso de Engenharia de Materiais**

---

**Organização:** Coordenação do Curso de Engenharia de Materiais  
**Contato:** [secretariamateriaisufpel@gmail.com](mailto:secretariamateriaisufpel@gmail.com)  
**Última atualização:** 21/10/2025


In [ ]:
import pandas as pd
from IPython.display import display, HTML
import gspread  # <-- ADICIONADO
from google.oauth2.service_account import Credentials  # <-- ADICIONADO

# --- INÍCIO DA PARTE DE AUTENTICAÇÃO E LEITURA ---

# 1. Definir os escopos (o que queremos acessar)
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# 2. Definir o nome do arquivo de credenciais
# (Este arquivo será criado pelo GitHub Action a partir do "Secret")
creds_file = 'gcreds.json' 

# 3. Autenticar
creds = Credentials.from_service_account_file(creds_file, scopes=scopes)
client = gspread.authorize(creds)

# 4. Abrir a planilha e pegar os dados
# !!! IMPORTANTE: Substitua "Nome-da-Sua-Planilha" pelo nome exato da sua planilha no Google Sheets
# !!! IMPORTANTE: Substitua "Planilha1" se o nome da aba (guia) for diferente
try:
    google_sheet = client.open("planilha-2025-2-real") # (Verifique se este é o nome correto)
    aba = google_sheet.worksheet("Planilha1") # (Verifique se este é o nome correto)
    dados = aba.get_all_records()

    # 5. Converter para o DataFrame pandas
    planilha = pd.DataFrame(dados)

    # --- LINHAS DE DEBUG (COM A INDENTAÇÃO CORRETA) ---
    # Elas agora estão DENTRO do bloco try
    # print(f"DEBUG: Lidas {len(planilha)} linhas da planilha.")
    # print(planilha.head())
    # --------------------------------------------------

except gspread.exceptions.SpreadsheetNotFound:
    print("ERRO: Planilha não encontrada. Verifique o nome da planilha no Google Sheets.")
    raise
except gspread.exceptions.WorksheetNotFound:
    print("ERRO: Aba da planilha não encontrada. Verifique o nome da aba (ex: 'Planilha1').")
    raise
except Exception as e:
    print(f"Ocorreu um erro ao acessar o Google Sheets: {e}")
    raise

# --- FIM DA PARTE DE AUTENTICAÇÃO E LEITURA ---

# O RESTO DO SEU CÓDIGO CONTINUA IGUAL A PARTIR DAQUI
# Mapas auxiliares
dias_da_semana = {'2': 'segunda-feira', '3': 'terça-feira', '4': 'quarta-feira', '5': 'quinta-feira', '6': 'sexta-feira'}
horarios_turno = {
    '1': ['08:00-08:50', '08:50-09:40', '10:00-10:50', '10:50-11:40', '11:40-12:30'],
    '2': ['13:30-14:20', '14:20-15:10', '15:10-16:00', '16:00-16:50', '17:10-18:00', '18:00-18:50'],
    '3': ['19:00-19:50', '19:50-20:40', '20:40-21:30', '21:30-22:20', '22:20-23:10']
}

# Mapas auxiliares
dias_da_semana = {'2': 'segunda-feira', '3': 'terça-feira', '4': 'quarta-feira', '5': 'quinta-feira', '6': 'sexta-feira'}
horarios_turno = {
    '1': ['08:00-08:50', '08:50-09:40', '10:00-10:50', '10:50-11:40', '11:40-12:30'],
    '2': ['13:30-14:20', '14:20-15:10', '15:10-16:00', '16:00-16:50', '17:10-18:00', '18:00-18:50'],
    '3': ['19:00-19:50', '19:50-20:40', '20:40-21:30', '21:30-22:20', '22:20-23:10']
}

# Função para gerar tabela HTML
def gerar_tabela_html(tabela_horarios, titulo):
    html = f'<h3 style="text-align: center; font-size: 1.5em; font-weight: bold;">{titulo}</h3>'
    html += '<table border="1" cellpadding="5" cellspacing="0" style="border-collapse: collapse; text-align: center; width: 100%;">'
    html += '<thead><tr><th>Horário</th><th>Segunda-feira</th><th>Terça-feira</th><th>Quarta-feira</th><th>Quinta-feira</th><th>Sexta-feira</th></tr></thead><tbody>'
    
    for index, row in tabela_horarios.iterrows():
        if row['horário'] == '---':
            html += '<tr><td colspan="6" style="background-color: #e0e0e0; font-weight: bold;">Intervalo entre Turnos</td></tr>'
        else:
            html += '<tr>' + ''.join(f'<td>{item}</td>' for item in row) + '</tr>'
    
    html += '</tbody></table>'
    return html

# Função para gerar e exibir tabela de horários
def gerar_tabela_horarios(df, titulo):
    horarios = pd.DataFrame(df, columns=['horário', 'dia', 'disciplina'])
    tabela_pivot = horarios.pivot_table(index='horário', columns='dia', values='disciplina', aggfunc=lambda x: '<br>'.join(x)).fillna('')

    # Garante todos os dias da semana
    for dia in dias_da_semana.values():
        if dia not in tabela_pivot.columns:
            tabela_pivot[dia] = ''
    tabela_pivot = tabela_pivot[['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira']]

    # Horários completos com intervalos
    turnos_usados = set()
    for linha in df:
        if linha[0] in horarios_turno['1']: turnos_usados.add('1')
        elif linha[0] in horarios_turno['2']: turnos_usados.add('2')
        elif linha[0] in horarios_turno['3']: turnos_usados.add('3')

    horarios_completos = []
    for i, turno in enumerate(sorted(turnos_usados)):
        horarios_completos.extend(horarios_turno[turno])
        if i < len(turnos_usados) - 1:
            horarios_completos.append('---')
    
    tabela_final = pd.DataFrame({'horário': horarios_completos})
    tabela_final = pd.merge(tabela_final, tabela_pivot, on='horário', how='left').fillna('')

    display(HTML(gerar_tabela_html(tabela_final, titulo)))

# Dicionários para guardar os dados
semestres_pares = {}
reofertas = []
optativas = []

# Processamento das linhas da planilha
for _, row in planilha.iterrows():
    semestre = row['semestre']
    if pd.isna(semestre):
        continue
    
    # Converte o semestre para int para a checagem de par/ímpar (caso venha como string)
    try:
        semestre = int(semestre)
    except ValueError:
        continue # Pula se o semestre não for um número

    codigo = row.get('codigo', '')
    nome = row.get('disciplina', '')
    turma = row.get('turma', '')
    prof = row.get('professor', '')

    horarios_disciplina = []
    for i in range(1, 7):
        horario_col = f'horario {i}'
        sala_col = f'sala {i}'
        
        horario_val = row.get(horario_col) # Pega o valor da coluna (ex: 'horario 1')

        # --- ESTA É A CORREÇÃO PRINCIPAL ---
        # Checa se o valor não é nulo (NaN) E se não é uma string vazia
        if pd.notna(horario_val) and str(horario_val).strip() != '':
            
            # Se passou no teste, continuamos como antes:
            try:
                cod_horario = str(int(horario_val)) # Agora esta linha é segura
            except ValueError:
                continue # Pula este horário se o valor não for um número (ex: "a definir")

            dia = dias_da_semana.get(cod_horario[0], '')
            turno = cod_horario[1]
            aula_idx = int(cod_horario[2]) - 1
            
            # Checagem de segurança para índice de aula
            if turno in horarios_turno and 0 <= aula_idx < len(horarios_turno[turno]):
                hora = horarios_turno[turno][aula_idx]
            else:
                continue # Pula este horário se o código for inválido

            sala = row.get(sala_col, '')
            
            # Verificação de sala melhorada (checa NaN, string vazia, ou 'nan' como texto)
            if pd.isna(sala) or str(sala).strip().lower() == 'nan' or str(sala).strip() == '':
                sala = 'sala indefinida'
            else:
                sala = f'sala {sala}'

            texto = f'{codigo} - {nome} {turma} (Prof. {prof} - {sala})'
            horarios_disciplina.append([hora, dia, texto])

    if not horarios_disciplina:
        continue

    # Classificação por tipo
    if semestre % 2 == 0 and semestre < 10:
        if semestre not in semestres_pares:
            semestres_pares[semestre] = []
        semestres_pares[semestre].extend(horarios_disciplina)
    elif semestre % 2 == 1 and semestre < 10:
        reofertas.extend(horarios_disciplina)
    elif semestre == 88:
        optativas.extend(horarios_disciplina)

# Geração das tabelas
for semestre, lista in sorted(semestres_pares.items()):
    gerar_tabela_horarios(lista, f'{semestre}º semestre')

if reofertas:
    gerar_tabela_horarios(reofertas, 'Reofertas')

if optativas:
    gerar_tabela_horarios(optativas, 'Optativas')
